<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Forex_Trader_Bot_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta

import datetime
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas_ta as ta

# Gather Data
#symbol = "USDJPY=X"
#symbol ="EURUSD=X"
#symbol = "GBPUSD=X"
symbol = "AUDUSD=X"

start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df = yf.download(symbol, start=start_date, end=end_date, interval="1d")

# Calculate technical indicators
df["RSI"] = ta.rsi(df["Close"])
df[["MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist"]] = ta.macd(df["Close"])
df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"])

# Drop NaN values
df.dropna(inplace=True)

# Preprocess Data
df_scaler = MinMaxScaler()
input_features = ["Close", "RSI", "MACD_12_26", "MACD_12_26_Signal", "MACD_12_26_Hist", "ATR"]
df[input_features] = df_scaler.fit_transform(df[input_features])

close_scaler = MinMaxScaler()
df['Close'] = close_scaler.fit_transform(df[['Close']])

# Reset index to ensure sequential integers
df.reset_index(drop=True, inplace=True)

# Split Data
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length - 1):  # Adjusted for the range
        sequence = data[i : i + seq_length].values
        label = data["Close"].iloc[i + seq_length]  # Use the 'Close' value at i + seq_length as the label
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 60
train_x, train_y = create_sequences(train_data[input_features], seq_length)  # Provide only the input features
test_x, test_y = create_sequences(test_data[input_features], seq_length)  # Provide only the input features

# Build Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer="adam", loss="mean_squared_error")

# Train Model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Make Predictions
predictions = model.predict(test_x)
predictions = close_scaler.inverse_transform(predictions)

# Evaluate Model
mse = mean_squared_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
mae = mean_absolute_error(predictions, close_scaler.inverse_transform(test_y.reshape(-1, 1)))
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Save Model
model.save("forex_trading_bot_model.keras")

# Make a prediction for the next day
next_day_sequence = np.expand_dims(test_x[-1], axis=0)
next_day_prediction = model.predict(next_day_sequence)
next_day_price = close_scaler.inverse_transform(next_day_prediction)[0][0]

# Print the predicted price for the next day
print("Predicted price for the next day:", next_day_price)

# Determine the action based on the predicted price
if next_day_price > df["Close"].iloc[-1]:
    print("Action: Buy")
elif next_day_price < df["Close"].iloc[-1]:
    print("Action: Sell")
else:
    print("Action: Hold")

# Create a baseline model that predicts the last known price as the next day's price
baseline_predictions = np.roll(test_x[:,-1,0], -1)  # Shift the last known prices one step forward
baseline_mse = mean_squared_error(test_y[:-1], baseline_predictions[:-1])
baseline_mae = mean_absolute_error(test_y[:-1], baseline_predictions[:-1])

# Calculate the Mean Squared Error (MSE) and Mean Absolute Error (MAE) for the LSTM model
lstm_mse = mean_squared_error(test_y, predictions)
lstm_mae = mean_absolute_error(test_y, predictions)

# Print the comparison results
print("\nBaseline Model:")
print("MSE:", baseline_mse)
print("MAE:", baseline_mae)

print("\nLSTM Model:")
print("MSE:", lstm_mse)
print("MAE:", lstm_mae)

# Compare the LSTM model's performance with the baseline model's performance
if lstm_mse < baseline_mse and lstm_mae < baseline_mae:
    print("\nConclusion: The LSTM model's performance is better than the baseline model. Consider using LSTM predictions for trading forex.")
    if next_day_price > df["Close"].iloc[-1]:
        print("Action: Buy")
    elif next_day_price < df["Close"].iloc[-1]:
        print("Action: Sell")
    else:
        print("Action: Hold")
elif lstm_mse > baseline_mse and lstm_mae > baseline_mae:
    print("\nConclusion: The baseline model's performance is better than the LSTM model. Caution advised.")
else:
    print("\nConclusion: The LSTM model's performance is similar to the baseline model. Use additional analysis for decision-making.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=e0aff098a6845d36ff1bc21b1c255b4f83175ece9d314f1eb26901a05883d507
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
87/87 [==============================] - 11s 64ms/step - loss: 0.0268
Epoch 2/10
87/87 [==============================] - 7s 83ms/step - loss: 8.0093e-04
Epoch 3/10
87/87 [==============================] - 5s 63ms/step - loss: 5.1906e-04
Epoch 4/10
87/87 [==============================] - 7s 78ms/step - loss: 4.7907e-04
Epoch 5/10
87/87 [==============================] - 6s 64ms/step - loss: 4.0190e-04
Epoch 6/10
87/87 [==============================] - 7s 79ms/step - loss: 3.9263e-04
Epoch 7/10
8